In [ ]:
# This is the template for the submission. If you want, you can develop your algorithm in a regular Python script and copy the code here for submission.

# Team members (e-mail, legi):
# chozhang@student.ethz.ch, 22-945-562
# minghli@student.ethz.ch, 22-953-293
# changli@student.ethz.ch, 22-944-474


In [1]:
from typing import *
import os
import sys
curr_environ = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', 'Localhost')
if curr_environ != 'Localhost':
    sys.path.append('/kaggle/input/mobile-health-2023-path-detection')
    input_dir = '/kaggle/input/mobile-health-2023-path-detection'
else:
    input_dir = os.path.abspath('')


In [2]:
import numpy as np
import pandas as pd

# progress bar
from tqdm import tqdm
from time import time

# utilty
from Lilygo.Recording import Recording
from Lilygo.Dataset import Dataset
from os import listdir
from os.path import isfile, join

# plotting
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# for signal processing and calculations
from scipy import signal
from scipy.integrate import simpson
from scipy import stats

from sklearn import tree

# dump and load model
from joblib import dump, load

# for tuning parameters
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# for skeleton
from sklearn.base import BaseEstimator


In [4]:
### signal processing functions ###
def parse(signal, ds_freq: float = 20.0, zero_mean: bool = False):
    """downsampling the signal to specific frequency ds_freq, and make the data
     with zero mean if zero_mean is True"""
    ori_time_seq = np.array(signal.timestamps)
    ori_value_seq = np.array(signal.values)
    if zero_mean:
        ori_value_seq = ori_value_seq - np.mean(ori_value_seq)
    dt = 1./ds_freq
    time_seq = np.arange(start=np.min(ori_time_seq),
                         stop=np.max(ori_time_seq),
                         step=dt)
    value_seq = np.interp(time_seq, ori_time_seq, ori_value_seq)
    return value_seq

In [5]:
# do not run this often
train_dir = '/kaggle/input/mobile-health-2023-path-detection/data/train/'
all_train_f = [os.path.join(train_dir, f) for f in os.listdir(train_dir)]
all_train_f.sort()


In [6]:
# watch location train and test
train_f, test_f = train_test_split(all_train_f, test_size=0.4, random_state=0)


In [7]:
def encode_feat_vec(sm):
    """ return a list containing feature vectors of all the 10s non-overlapping windows"""
    WINDOW = 2000  # 10s window
    feat_vecs = []
    df1s = []
    df2s = []
    dfb2s = []
    sf = 50.0
    for i in range(0, len(sm) - WINDOW, WINDOW):
        data = sm[i:i+WINDOW]
        low1 = 0.3
        high1 = 15
        low2 = 0.6
        high2 = 2.5
        nperseg = 2 / low1 * sf
        freqs, psd = signal.welch(data, sf, nperseg=nperseg)
        BAND1 = np.logical_and(freqs >= low1, freqs <= high1)
        BAND2 = np.logical_and(freqs >= low2, freqs <= high2)
        freq_res = freqs[1] - freqs[0]
        # compuate total power
        total_power = simpson(psd[BAND1], dx=freq_res)

        # compute dominant frequencies for BAND1, BAND2
        idx_p1 = psd[BAND1].argsort()[-1]
        df1 = low1 + freq_res * idx_p1
        pdf1 = simpson(psd[BAND1][idx_p1:idx_p1+2], dx=freq_res)
        df1s.append(df1)

        idx_p2 = psd[BAND1].argsort()[-2]
        df2 = low1 + freq_res * idx_p2
        pdf2 = simpson(psd[BAND1][idx_p2:idx_p2+2], dx=freq_res)
        df2s.append(df2)

        idx_b2 = psd[BAND2].argsort()[-1]
        dfb2 = low2 + freq_res * idx_b2
        pdfb2 = simpson(psd[BAND2][idx_b2:idx_b2+2], dx=freq_res)
        dfb2s.append(dfb2)

        ratio_pdf1_tot = pdf1 / total_power

        if i != 0:
            ratio_curr_prev_df1 = df1s[-1] / df1s[-2]
            ratio_curr_prev_df2 = df2s[-1] / df2s[-2]
            ratio_curr_prev_dfb2 = dfb2s[-1] / dfb2s[-2]
        else:
            ratio_curr_prev_df1, ratio_curr_prev_df2, ratio_curr_prev_dfb2 = \
                1.0, 1.0, 1.0

        R1 = simpson(psd[freqs <= 3], dx=freq_res) / total_power
        R2 = simpson(psd[freqs > 3], dx=freq_res) / total_power

        feat_vec = [
            np.mean(data),
            np.std(data),
            df1, pdf1, df2, pdf2, dfb2, pdfb2,
            ratio_pdf1_tot,
            ratio_curr_prev_df1, ratio_curr_prev_df2, ratio_curr_prev_dfb2,
            R1, R2]
        feat_vecs.append(feat_vec)
    return feat_vecs


In [8]:
def compute_sm(ax, ay, az, normalize=False):
    sm = np.sqrt(np.sum(np.square([ax, ay, az]), axis=0))
    if normalize:
        return sm - np.mean(sm)
    else:
        return sm


In [9]:
def parse_accelerometer(trace: Recording, use_zc=True):
    if use_zc:
        ax = parse(trace.data['ax'])
        ay = parse(trace.data['ay'])
        az = parse(trace.data['az'])
    else:
        ax = trace.data['ax'].values
        ay = trace.data['ay'].values
        az = trace.data['az'].values
    return ax, ay, az


In [ ]:
X_train = []
y_train = []

for f in tqdm(train_f):
    t = Recording(f, no_labels=False, mute=True)
    ax, ay, az = parse_accelerometer(t, use_zc=False)
    label = t.labels['board_loc']
    sm = compute_sm(ax, ay, az, normalize=True)
    fv = encode_feat_vec(sm)
    X_train.extend(fv)
    y_train.extend([label] * len(fv))


In [ ]:
# decision tree version
class WatchLocDetector(BaseEstimator):
    def __init__(self):
        self.clf = tree.DecisionTreeClassifier()

    def fit(self, X, y):
        selector = np.array([fv[1] > 0.1 for fv in X])
        _X = X[selector]
        if isinstance(y, int):
            _y = [y] * len(_X)
        else:
            _y = y[selector]
        self.clf.fit(_X, _y)

    def score(self, X, y, sample_weight=None):
        from sklearn.metrics import accuracy_score
        return accuracy_score(y, self.predict(X), sample_weight=sample_weight)

    def predict_fv(self, X):
        win_res = self.clf.predict(X)
        return stats.mode(win_res).mode[0]

    def predict(self, traces):
        # assume array
        if hasattr(traces, '__len__'):
            res = np.zeros(len(traces), dtype=int)
            _traces = traces
        else:
            res = np.zeros(1, dtype=int)
            _traces = [traces]
        i = 0
        for trace in _traces:
            if isinstance(trace, Recording):
                ax, ay, az = parse_accelerometer(trace)
                sm = compute_sm(ax, ay, az, normalize=True)
                feat_vec = encode_feat_vec(sm)
            else:
                feat_vec = trace
            win_res = self.clf.predict(feat_vec)
            res[i] = stats.mode(win_res).mode[0]
            i += 1
        if len(res) == 1:
            return res[0]
        else:
            return res


In [ ]:
# activity recognition train and test
X_train_act = []
y_train_act = []
test_f_act = []
for f in tqdm(all_train_f):
    t = Recording(f, no_labels=False, mute=True)
    # use homogeneous traces to train
    if len(t.labels['activities']) == 1:
        ax, ay, az = parse_accelerometer(t, use_zc=False)
        sm = compute_sm(ax, ay, az, normalize=True)
        fv = encode_feat_vec(sm)
        X_train_act.extend(fv)
        y_train_act.extend(t.labels['activities'] * len(fv))
    else:
        test_f_act.append(f)

In [ ]:
class ActivityRecognizer(BaseEstimator):
    def __init__(self, count_tresh=2):
        self.clf = tree.DecisionTreeClassifier()
        self.count_thresh = count_tresh

    def fit(self, X, y): # X is an array of windows
        selector = np.array([fv[1] > 0.1 for fv in X])
        _X = X[selector]
        if isinstance(y, int):
            _y = [y] * len(_X)
        else:
            _y = y[selector]
        self.clf.fit(_X, _y)

    def score(self, X, y, sample_weight=None):
        y_predict = self.predict(X)
        tot_score = 0
        for true, pred in zip(y, y_predict):
            # walk 1, run 2, cycle 3
            # 0.05, 0.05, 0.1
            true_v = [1 in true, 2 in true, 3 in true]
            pred_v = [1 in pred, 2 in pred, 3 in pred]
            score = int(true_v[0] == pred_v[0]) + \
                    int(true_v[1] == pred_v[1]) + \
                    int(true_v[2] == pred_v[2])
            score /= 4
            tot_score += score
        return tot_score / len(y_predict)

    def predict_fv(self, X):
        win_res = self.clf.predict(X)
        values, counts = np.unique(win_res, return_counts=True)
        return [values[j] 
                for j, count in enumerate(counts) if count >= self.count_thresh]

    def predict(self, traces):
        # assume array
        if hasattr(traces, '__len__'):
            res = [0] * len(traces)
            _traces = traces
        else:
            res = [0]
            _traces = [traces]
        i = 0
        for trace in _traces:
            if isinstance(trace, Recording):
                ax, ay, az = parse_accelerometer(trace, use_zc=False)
                sm = compute_sm(ax, ay, az, normalize=True)
                feat_vec = encode_feat_vec(sm)
            else:
                feat_vec = trace
            # will get an activity prediction for each window
            win_res = self.clf.predict(feat_vec) 
            values, counts = np.unique(win_res, return_counts=True)
            res[i] = [values[j] 
                      for j, count in enumerate(counts) if count >= self.count_thresh]
            i += 1
        return res


In [ ]:
watchloc_detector = WatchLocDetector()
watchloc_detector.fit(np.array(X_train), np.array(y_train))


In [ ]:
act_recognizer = ActivityRecognizer()
act_recognizer.fit(X_train_act, y_train_act)

In [ ]:
dump(watchloc_detector.clf, '/kaggle/working/group24_model_watchloc.joblib')

In [ ]:
dump(act_recognizer.clf, '/kaggle/working/group24_model_activity.joblib')

In [ ]:
from scipy.spatial.transform import Rotation

def madgwick_update(q, gyro, accel, mag, beta, dt):
    q = np.array([q[0], q[1], q[2], q[3]])

    f_g = np.array([2 * (q[1] * q[3] - q[0] * q[2]) - accel[0],
                    2 * (q[0] * q[1] + q[2] * q[3]) - accel[1],
                    2 * (0.5 - q[1]**2 - q[2]**2) - accel[2]])

    f_b = np.array([2 * (q[1] * q[2] + q[0] * q[3]) - mag[0],
                    2 * (q[0] * q[1] - q[2] * q[3]) - mag[1],
                    2 * (q[0]**2 + q[3]**2 - 0.5) - mag[2]])

    j_g = np.array([[-2 * q[2], 2 * q[3], -2 * q[0], 2 * q[1]],
                    [2 * q[1],  2 * q[0],  2 * q[3], 2 * q[2]],
                    [0,         -4 * q[1], -4 * q[2],0]])

    j_b = np.array([[2 * q[3],  2 * q[2],  2 * q[1], 2 * q[0]],
                    [2 * q[0],  -2 * q[1], -2 * q[2],2 * q[3]],
                    [-4 * q[0], -4 * q[3],  0,       0]])

    step_size = beta * dt
    q += step_size * (j_g.T @ f_g + j_b.T @ f_b)
    q /= np.linalg.norm(q)
    return q

class PathDetector(BaseEstimator):
    # hack labels via GPS, delete later
    def __init__(self):
        pass
    
    def extract(self,trace):
        data = trace.data
        fea_alti = self.get_fea_alti(data['altitude'])
        fea_ori = self.get_fea_ori(data['ax'],data['ay'],data['az'],data['gx'],data['gy'],data['gz'],data['mx'],data['my'],data['mz'])
        fea = fea_alti + fea_ori
        return fea

    # required
    def fit(self, data, labels):
        self.clf = RandomForestClassifier(min_samples_leaf=10, max_depth=15, max_features=12)
        self.clf.fit(data, labels)
        print('fit score', self.clf.score(data,labels))

    # required    
    def predict(self, feature):
        fea = [feature]
        res = self.clf.predict(fea)[0]
        return res
    
    def get_fea_alti(self, alti):
        t, alti = parse(alti)
        t = t[600:]
        alti = alti[600:]
        alti = lp_filter(alti, .998)
        alti = alti - np.min(alti)
        alti[alti>60] = 60
        alti /= 60.0
        t = (t-np.min(t)) / (np.max(t)-np.min(t))
        fea_alti = split_fea(t, alti, 5, lambda x: np.nanpercentile(x,50))
        return fea_alti
    
    def get_fea_ori(self, ax, ay, az, gx, gy, gz, mx, my, mz):
        deg2rad = np.pi/180
        t, ax = parse(ax)
        t, ay = parse(ay)
        t, az = parse(az)
        t, gx = parse(gx)
        t, gy = parse(gy)
        t, gz = parse(gz)
        t, mx = parse(mx)
        t, my = parse(my)
        t, mz = parse(mz)
        gx, gy, gz = gx * deg2rad, gy * deg2rad, gz * deg2rad
        acc = np.array([ax,ay,az])
        gyro = np.array([gx,gy,gz])
        mag = np.array([mx,my,mz])

        acc /= (1e-8 + np.linalg.norm(acc,ord=2,axis=0,keepdims=True))
        mag /= (1e-8 + np.linalg.norm(mag,ord=2,axis=0,keepdims=True))

        dt = t[1] - t[0]
        beta = 0.1  # Madgwick filter gain
        
        ref_acc_vector = np.array([[0, 0, 1],[0, 0, 1],[0, 0, 1]])  # a - g for a = 0
        rotation_matrix = Rotation.align_vectors(ref_acc_vector, [acc[:,0],acc[:,1],acc[:,2]])[0].as_matrix()
        q = Rotation.from_matrix(rotation_matrix).as_quat()
        
        quats = []
        yaws = []
        for i in range(len(t)):
            q = madgwick_update(q, gyro[:,i], acc[:,i], mag[:,i], beta, dt)
            quats.append(q)
            try:
                yaw = Rotation.from_quat(q).as_euler('xyz')[-1]
            except: 
                yaw = 0
            yaws.append(yaw)
        t, yaws = np.array(t[400:]), np.array(yaws[400:])
        yaws -= yaws[0]  # sometime can jump 2pi
        sinyaws, cosyaws = np.sin(yaws), np.cos(yaws)
        sinyaws, cosyaws = lp_filter(sinyaws, .97), lp_filter(cosyaws, .97)
        fea_sinyaw = split_fea(t, sinyaws, 10)
        fea_cosyaw = split_fea(t, cosyaws, 10)
        fea_ori = fea_sinyaw + fea_cosyaw
        return fea_ori

path_detector = PathDetector()

In [ ]:
dir_traces_train = '/kaggle/input/mobile-health-2023-path-detection/data/train'
filenames_train = [join(dir_traces_train, f) for f in listdir(dir_traces_train) if isfile(join(dir_traces_train, f))]
filenames_train.sort()
labels = []
features = []
for f in filenames_train:
    trace = Recording(f, no_labels=False, mute=True)
    label = trace.labels['path_idx']
    labels.append(label)
    fea = path_detector.extract(trace)
    features.append(fea)
    print(f, label, len(fea), end = '     \r')
    
labels = np.array(labels)
features = np.array(features)

In [ ]:
np.save('/kaggle/working/pathfea.npy', features)
np.save('/kaggle/working/pathlabels.npy', labels)
print(features.shape, labels.shape)

In [ ]:
features = np.load('pathfea.npy')
labels = np.load('pathlabels.npy')

# clf = RandomForestClassifier(min_samples_leaf=10, max_depth=15, max_features=12)
# scores = cross_val_score(clf, features, labels, cv=5, scoring='accuracy')
# print(scores, np.mean(scores), np.std(scores))
path_detector.fit(features, labels)
path_detector.predict(features[0])

In [ ]:
dump(path_detector.clf, '/kaggle/working/group24_model_pathindex.joblib')

In [ ]:
# Get the path of all traces and encode features
dir_traces = '/kaggle/input/mobile-health-2023-path-detection/data/test'
filenames = [join(dir_traces, f) for f in listdir(dir_traces) if isfile(join(dir_traces, f))]
filenames.sort()

pathdec_fea_test = []
for i, filename in enumerate(filenames):
    trace = Recording(filename, no_labels=True, mute=True)
    pathdec_fea_test.append(path_detector.extract(trace))
    print(filename, end = '   \r')
pathdec_fea_test = np.array(pathdec_fea_test)
print(pathdec_fea_test.shape)

np.save('/kaggle/working/group24_features_pathindex.npy', pathdec_fea_test)